In [44]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
import time
import pafy
from bs4 import BeautifulSoup
import requests
import re
import webbrowser
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image
from IPython.core.display import HTML 
import time
from ipywidgets import FloatProgress
from IPython.display import display
from textblob import TextBlob
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from IPython.display import Image
from nltk.tokenize import RegexpTokenizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import multiprocessing
import gensim.models.doc2vec
from gensim.models.phrases import Phrases, Phraser
import sklearn

In [77]:
corpus_df = pd.read_csv('ALL_rows_scraped.csv', index_col=0)

In [143]:
sources_labels = set([(row.source_id, row.label) for index,row in corpus_df.iterrows()])

In [144]:
sources_labels

{('associated-press', 'center'),
 ('breitbart-news', 'right'),
 ('cnn', 'left'),
 ('daily-mail', 'right'),
 ('fox-news', 'right'),
 ('msnbc', 'left'),
 ('national-review', 'right'),
 ('reuters', 'center'),
 ('the-american-conservative', 'right'),
 ('the-economist', 'center'),
 ('the-new-york-times', 'center'),
 ('the-wall-street-journal', 'center'),
 ('the-washington-times', 'center'),
 ('time', 'left'),
 ('vice-news', 'left')}

In [53]:
# corpus_df = corpus_df.head(1000)

In [172]:
sample_df = corpus_df.sample(5000)
sample_df = sample_df.reset_index()
# sample_df = sample_df.drop('index', axis=1)

In [173]:
sample_df.head(1)

,index,author,content,description,label,publishedAt,source_id,source_name,title,url,urlToImage,text,neg,neu,pos,compound,subjectivity
0,3259,none_provided,"JEFFERSON CITY, Mo. (AP) — Missouri Highway Pa...","JEFFERSON CITY, Mo. (AP) — Missouri Highway Pa...",center,2018-08-24T19:59:02Z,associated-press,Associated Press,"Missouri Highway Patrol leader to retire, lead...",https://apnews.com/5f182d21ff4f413cac3acb24d88...,https://bloximages.newyork1.vip.townnews.com/r...,"JEFFERSON CITY, Mo. (AP) Missouri Highway Pat...",0.019,0.952,0.029,0.0772,0.283889


In [174]:
x_train = sample_df.text

In [175]:
tokenized_train = [article.split() for article in x_train]
phrases = Phrases(tokenized_train)
bigram = Phraser(phrases)

In [176]:
# print(bigram[x_train[0].split()])

In [177]:
tokenizer = RegexpTokenizer('[A-Za-z]\w+')
data = list(sample_df.text)
labels = list(sample_df.label)
sources = list(sample_df.source_name)

tagged_data = [TaggedDocument(words=bigram[tokenizer.tokenize(word)], tags=[labels[index],sources[index] ,str(sources[index])+' '+str(labels[index])+' '+ str(index)]) for index, word in enumerate(data)]
    

In [188]:
tagged_data[5]

TaggedDocument(words=['As', 'Donald_Trump', 'threatens', 'further', 'tariffs_on', 'Chinese_imports', 'the', 'prospect_of', 'deal', 'is', 'receding', 'IT_IS', 'becoming_increasingly', 'likely', 'that', 'the', 'phoney', 'trade_war', 'between', 'America', 'and', 'China', 'will', 'develop', 'into', 'the', 'real', 'thing', 'On_June', 'th', 'the', 'Trump_administration', 'published', 'two', 'lists', 'of', 'Chinese', 'products', 'it', 'plans', 'to', 'hit', 'with', 'tariffs', 'of', 'worth', 'bn', 'in', 'The', 'first', 'will', 'come', 'into', 'force', 'on', 'July', 'th', 'The', 'Chinese', 'snapped', 'back', 'with', 'their_own', 'list', 'laying_out', 'retaliation', 'of', 'equal', 'size', 'Then', 'on', 'June', 'th', 'President_Donald', 'Trump', 'directed', 'Robert_Lighthizer', 'the', 'United_States', 'Trade_Representative', 'USTR', 'to', 'draw', 'up', 'further', 'list_of', 'products', 'worth', 'bn', 'that', 'would', 'face', 'tariffs', 'of', 'and', 'threatened', 'yet_another', 'covering', 'an_addi

In [178]:
assert gensim.models.doc2vec.FAST_VERSION > -1
max_epochs = 200
vec_size = 50
alpha = 0.025
cores = multiprocessing.cpu_count()

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=2,
                dm =0, workers=cores)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(sklearn.utils.shuffle(tagged_data),
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [196]:
model.most_similar('great')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('London', 0.5576285123825073),
 ('BOOKER', 0.5544752478599548),
 ('Vaticans', 0.5210408568382263),
 ('Ronald', 0.5157053470611572),
 ('exploring', 0.5055341124534607),
 ('thunderstorms', 0.5032579302787781),
 ('dont_remember', 0.49266496300697327),
 ('Chiang', 0.4885493516921997),
 ('affixed', 0.48729604482650757),
 ('abstain', 0.4819973111152649)]

In [194]:
# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('Fox News')
print(similar_doc)

# print(model.docvecs['1']) #print doc vector

[('The American Conservative', 0.9369813799858093), ('Time', 0.9279861450195312), ('National Review right 4512', 0.5756717920303345), ('National Review right 1101', 0.5621157288551331), ('National Review right 910', 0.5481550693511963), ('MSNBC left 2228', 0.5106648802757263), ('CNN left 3845', -0.013068556785583496), ('CNN left 2300', -0.022524692118167877), ('MSNBC left 1541', -0.0862095057964325), ('Associated Press center 1754', -0.1268918812274933)]


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [190]:
testing_math = model.most_similar(positive=['Alex_Jones'], negative=['right'])
print(similar_doc)

[('Vice News', 0.9902253150939941), ('center', 0.9772250056266785), ('The Economist', 0.9745921492576599), ('The New York Times', 0.9671074151992798), ('right', 0.9652225971221924), ('Reuters', 0.9580557346343994), ('Daily Mail', 0.956326961517334), ('Breitbart News', 0.9547126293182373), ('Associated Press', 0.9361096024513245), ('National Review', 0.9012750387191772)]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [179]:
# {('associated-press', 'center'),
#  ('breitbart-news', 'right'),
#  ('cnn', 'left'),
#  ('daily-mail', 'right'),
#  ('fox-news', 'right'),
#  ('msnbc', 'left'),
#  ('national-review', 'right'),
#  ('reuters', 'center'),
#  ('the-american-conservative', 'right'),
#  ('the-economist', 'center'),
#  ('the-new-york-times', 'center'),
#  ('the-wall-street-journal', 'center'),
#  ('the-washington-times', 'center'),
#  ('time', 'left'),
#  ('vice-news', 'left')}